### Fractional Cover Sentinel 2a/b
Created by Amos Bennett.<br>
Last Updated 15 Sep 20. <br>

__Updates:__ 

This script loads a pre and postfire Landsat 8 fractional cover product, and pansharpens the product using a pseudo Sentinel 2 panchromatic band in order to generate a Sentinel 2 equivalent fractional cover product. This will then be used in additional classifications with a dNBR in order to improve the accuracy of a fire severity assessment.

Hanging on to this code for the random sampling functionality built in.

In [ ]:
import sys
import datacube
from datacube.helpers import write_geotiff
from datacube.utils import cog
import pandas as pd
import xarray as xr
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

sys.path.append("/home/554/ab4513/dea-notebooks/Scripts")
import dea_datahandling
from dea_datahandling import load_ard
from dea_plotting import rgb
from dea_plotting import display_map
from dea_plotting import map_shapefile
from dea_bandindices import calculate_indices

In [ ]:
print(datacube.__version__)

In [ ]:
dc = datacube.Datacube(app="fractionalCover")

In [ ]:
# Set the central latitude and longitude
central_lat = -35.783333
central_lon = 148.016667
crs = 'EPSG:32755'

# Canberra 148.6547665°E 35.5655761°S
# Kosciuszko 148.3517111°E 36.1864717°S
# Tumbarumba 148.016667°E 35.783333°S

# 0.1° approximately equal to 11.1km distance.

# Set the buffer to load around the central coordinates (even numbers such as 0.2, 1.0, 2.2 etc) in degrees (lat, lon)
buffer = 0.6

# Compute the bounding box for the study area
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

# display_map(x=study_area_lon, y=study_area_lat, margin=-0.2)

Due to the way that Landsat 8 fractional cover scenes are processed, the fractional cover products cannot be loaded across a wide timespan and statistically calculated with a mean or median. As such, only a few scenes are loaded by limiting the date range and selected the 'max' pixel values across the extent to mitigate the issue of NaN or 0 values distorting the dataset.

In [ ]:
# Key dates to load pre and postfire Landsat 8 FC scenes
prefire_start = '2019-12-21'
prefire_end = '2019-12-23'
postfire_start = '2020-01-07'
postfire_end = '2020-01-08'

The prefire Landsat 8 fractional cover product is retrieved using the normal dc.load() and query functions. Because the ls8_fc_albers is in an albers projection, the output coordinate reference system is specified to match UTM zone 55S.

In [ ]:
query = {"x": (central_lon - buffer, central_lon + buffer),
         "y": (central_lat - buffer, central_lat + buffer),
         "time": (prefire_start, prefire_end),
         "output_crs": "EPSG:32755",
         "resolution": (-25, 25)}

prefire_ls8fc_ds = dc.load(product="ls8_fc_albers",
                           measurements=['BS'],
                           **query)

prefire_ls8fc_ds

Code in contigency used to visually assess which images and dates should be used.

As described above, the prefire ls8fc image is calculated using a 'max' statistical calculation. The prefire fractional cover image is then plotted to quickly check for any unusual or glaring artefacts.

In [ ]:
prefire_ls8fc_image = prefire_ls8fc_ds.max(dim='time')
rgb(prefire_ls8fc_image, bands=["BS", "PV", "NPV"])

Red = Bare, Green = Photosynthetic, Blue = Non-Photosynthetic

In [ ]:
query = {"x": (central_lon - buffer, central_lon + buffer),
         "y": (central_lat - buffer, central_lat + buffer),
         "time": (postfire_start, postfire_end),
         "output_crs": "EPSG:32755",
         "resolution": (-25, 25)}

postfire_ls8fc_ds = dc.load(product="ls8_fc_albers",
                            measurements=['BS'],
                            **query)

postfire_ls8fc_ds

Code in contigency used to visually assess which images and dates should be used.

Postfire ls8 fractional cover image is also generated using the 'max' statistical calcuation. It becomes glaringly obvious that much of the biomass in the scene was burnt given that the red is a measure of bare soil as a percentage of a pixel indicating significant bare ground exposure after the fire (due to loss in canopy cover, biomass etc).

In [ ]:
postfire_ls8fc_image = postfire_ls8fc_ds.max(dim='time')
rgb(postfire_ls8fc_image, bands=["BS", "PV", "NPV"])

The next bit of code is used to randomly sample points across the pre and postfire fractional cover images in the event that data can be used to train a classifier to 'infer' a fractional cover product from Sentinel 2 ARD.

In [ ]:
prefire_datapoints = pd.DataFrame(columns=['y','x','crs','bs','pv','npv'])

for i in range(0,500):
    y = np.random.randint(0,5370, size=1)
    x = np.random.randint(0,4394, size=1)
    bs_df = prefire_ls8fc_image.BS[y,x].to_dataframe()
    pv_df = prefire_ls8fc_image.PV[y,x].to_dataframe()
    npv_df = prefire_ls8fc_image.NPV[y,x].to_dataframe()
    row = [bs_df.index[0][0], bs_df.index[0][1], bs_df.spatial_ref[0], bs_df.BS[0], pv_df.PV[0], npv_df.NPV[0]]
    series = pd.Series(row, index = prefire_datapoints.columns)
    prefire_datapoints = prefire_datapoints.append(series, ignore_index=True)

In [ ]:
postfire_datapoints = pd.DataFrame(columns=['y','x','crs','bs','pv','npv'])

for i in range(0,500):
    y = np.random.randint(0,5370, size=1)
    x = np.random.randint(0,4394, size=1)
    bs_df = postfire_ls8fc_image.BS[y,x].to_dataframe()
    pv_df = postfire_ls8fc_image.PV[y,x].to_dataframe()
    npv_df = postfire_ls8fc_image.NPV[y,x].to_dataframe()
    row = [bs_df.index[0][0], bs_df.index[0][1], bs_df.spatial_ref[0], bs_df.BS[0], pv_df.PV[0], npv_df.NPV[0]]
    series = pd.Series(row, index = postfire_datapoints.columns)
    postfire_datapoints = postfire_datapoints.append(series, ignore_index=True)

In [ ]:
prefire_datapoints_gdf = gpd.GeoDataFrame(prefire_datapoints, 
                                          geometry=gpd.points_from_xy(prefire_datapoints.x, prefire_datapoints.y), 
                                          crs={'init': 'epsg:32755'})
postfire_datapoints_gdf = gpd.GeoDataFrame(postfire_datapoints, 
                                           geometry=gpd.points_from_xy(postfire_datapoints.x, postfire_datapoints.y), 
                                           crs={'init': 'epsg:32755'})

Example of how to change the crs of a geodataframe if required.

In [ ]:
# Key dates to load Sentinel-2 ARD (as per dNBR based on statistical analysis)
prefire_start = '2019-11-01'
prefire_end = '2020-01-06'
postfire_start = '2020-01-07'
postfire_end = '2020-05-01'

Load the prefire Sentinel 2 images into an x-array. Because the NCI limits memory allocations, the x-array dataset will be dask chunked by x and y coordinates (500 x 500 x t). This ensures that all processing occurs within the memory allocation.